# Mortgage 30-years Rate
The Mortgage 30-years rate was retrieved from the FRED website (https://fred.stlouisfed.org/series/MORTGAGE30US).

In [65]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np

In [4]:
# URL to read and get data from
df = pd.read_csv('../.data/MORTGAGE30US.csv')
df.rename(columns={'observation_date': 'dt', 'MORTGAGE30US':'mtgage30us'}, inplace=True)

# Make columns 'dt' as datetime
df['dt'] = pd.to_datetime(df['dt'])

# Set data to index
df.set_index('dt', inplace=True)

In [5]:
# Resample data from weekly to monthly
ts = df.resample(rule='ME').last().reset_index()

ts

,dt,mtgage30us
0,1971-04-30,7.29
1,1971-05-31,7.46
2,1971-06-30,7.54
3,1971-07-31,7.69
4,1971-08-31,7.69
...,...,...
641,2024-09-30,6.08
642,2024-10-31,6.72
643,2024-11-30,6.81
644,2024-12-31,6.85


In [6]:
# Save to data folder
ts.to_csv('../.data/mortgage_rates.csv', index=False)

## Get Inflation Data
The data can be retrieved from the website: https://www.usinflationcalculator.com/inflation/historical-inflation-rates/

In [3]:
# Scrape data from URL
url = 'https://www.usinflationcalculator.com/inflation/historical-inflation-rates/'
dfi = pd.read_html(url)[0]

In [4]:
dd = dfi.copy()

In [41]:
dfi = dd.copy()

# Slice for years after 1970
dfi = dfi.query("Year > 1970")

# Adjust columns names from month name to numbers
dfi.rename(columns={'Year':'yr',
                    'Jan':1, 'Feb':2, 'Mar':3,'Apr':4,'May':5,'Jun':6,
                    'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12 }, 
                    inplace=True)

dfi.head(2)


,yr,1,2,3,4,5,6,7,8,9,10,11,12
57,1971,5.3,5.0,4.7,4.2,4.4,4.6,4.4,4.6,4.1,3.8,3.3,3.3
58,1972,3.3,3.5,3.5,3.5,3.2,2.7,2.9,2.9,3.2,3.4,3.7,3.4


In [ ]:
# Drop last row, because it is all NAs (2025 data)
dfi.drop(111, axis=0, inplace=True)

# Drop average column
dfi.drop('Ave', axis=1, inplace=True)

# Slice for years after 1970
dfi = dfi.query("Year > 1970")

# Adjust columns names from month name to numbers
dfi.rename(columns={'Year':'yr',
                    'Jan':1, 'Feb':2, 'Mar':3,'Apr':4,'May':5,'Jun':6,
                    'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12 }, 
                    inplace=True)

In [42]:
# Transform to long version
dfi = (dfi
       .melt(id_vars=['yr'],
            var_name='mth',
            value_vars=dfi.drop('yr', axis=1),
            value_name='inflation')
        .sort_values(by=['yr','mth'])
    )

# View final result
dfi.head(3)

,yr,mth,inflation
0,1971,1,5.3
54,1971,2,5.0
108,1971,3,4.7


In [68]:
# Adjust data types
dfi['yr'] = dfi['yr'].astype(str)
dfi['mth'] = dfi['mth'].astype(str)
dfi['inflation'] = dfi['inflation'].astype(float)

# Create date column
dfi['dt'] = pd.to_datetime(dfi['yr'] + '-' + dfi['mth']) + MonthEnd(1)

yr                   object
mth                  object
inflation           float64
dt           datetime64[ns]
dtype: object


In [71]:
dfi = dfi.reindex(columns=['dt', 'inflation'])
print(dfi.dtypes)

dt           datetime64[ns]
inflation           float64
dtype: object


In [72]:
# Save to data folder
dfi.to_csv('../.data/inflation_rates.csv', index=False)